In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MaxAbsScaler,MinMaxScaler
from imblearn.pipeline import  Pipeline as imbPipeline,make_pipeline
from pandas_profiling import ProfileReport
#Вывод неограниченного числа столбцов
pd.options.display.max_columns = None
# pd.set_option('display.max_columns', None)
#Вывод неограниченного числа строк
pd.options.display.max_rows = None
#Кол-во символов для отображения
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

**Цель:** в результате, необходимо предсказать вероятности **положительного** класса для данных из тестовой выборки.
<p><strong><font color='red'>Ограничения:</font></strong> качество решения оценивается по стандартной метрике ROC-AUC.</p>

In [2]:
def read_file (file_name):
    """
    Параметры:
    _ _ _ _ _
    file_name: str
               название файла с расширением типа 'train.csv"""
    data = pd.read_csv(file_name,header=None)
    print ('\033[1mРазмер набора данных:\033[0m', data.shape)
    print('\033[1mСтолбцы с Пустыми значений:\033[0m', data.isnull().sum()[data.isnull().sum()>0])
    print('\033[1mТипы данных в наборе:\033[0m', set(data.dtypes))
    return data

In [3]:
train = read_file('train.csv')
train.head()

Размер набора данных: (10000, 30)
Столбцы с Пустыми значений: Series([], dtype: int64)
Типы данных в наборе: {dtype('float64')}


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,-220.530530,-70.197440,119.035181,20.711737,-6.152986,52.225051,-23.230903,-166.521871,-41.571463,2.620401,271.292251,2.695313,44.529272,12.998666,25.157748,2.192931,3.438528,-81.684213,49.566982,-36.818566,16.936420,46.543585,-80.962171,-146.652218,10.899085,4.370985,4.060272,40.680890,-37.942655,33.838225
1,-8.536541,-8.305435,-117.828269,-28.588333,14.223240,-42.087807,-45.538664,58.898976,27.749744,2.545916,-97.233793,-4.899233,-104.382283,2.867261,-128.433975,2.280566,3.412432,31.501910,-47.950176,103.060202,57.027508,121.304258,31.507396,88.282912,66.766185,-4.363974,-5.791376,-32.101939,-52.255449,-4.826111
2,126.229973,133.463504,-105.511797,-149.110267,-3.673355,-104.302244,5.815395,100.838385,-46.240211,0.141700,194.054804,-3.134316,-145.510845,86.972373,-126.200888,0.598976,3.351103,51.084960,141.443443,-150.020696,4.819849,88.905713,51.475105,-73.257358,108.947287,8.567240,0.116269,-117.205053,-107.835928,-93.990332
3,369.571563,11.850181,-299.969407,29.371721,-3.457523,-115.901854,159.134323,-149.741411,-108.847522,0.078277,-215.802195,-1.724561,-128.050861,-99.997710,138.824836,0.460472,1.891893,-73.956562,-116.557681,310.894782,80.930568,-166.476192,-73.882682,108.222355,19.573192,-2.242024,0.515601,-25.218215,121.674228,-16.877368
4,-99.563708,-85.166292,-73.363391,-35.357907,5.437025,-54.892519,56.430532,-49.237377,-50.054608,2.586612,-115.997344,-0.058934,97.666751,36.416791,-124.206264,0.638184,3.989474,-23.264695,-92.966637,121.929801,31.682235,209.163125,-22.847302,-62.067337,58.109167,2.862082,0.920341,-132.875142,3.492953,-15.799596


In [4]:
train_target = read_file('train-target.csv')
print(train_target[0].value_counts(),'\n',round(train_target[0].value_counts()/train_target.shape[0], 2))
train_target.head(2)
train_target.columns = ['target']

Размер набора данных: (10000, 1)
Столбцы с Пустыми значений: Series([], dtype: int64)
Типы данных в наборе: {dtype('int64')}
1    5051
0    4949
Name: 0, dtype: int64 
 1    0.51
0    0.49
Name: 0, dtype: float64


In [5]:
train = train.merge(train_target, how ='left',right_index=True, left_index=True)
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,target
0,-220.530530,-70.197440,119.035181,20.711737,-6.152986,52.225051,-23.230903,-166.521871,-41.571463,2.620401,271.292251,2.695313,44.529272,12.998666,25.157748,2.192931,3.438528,-81.684213,49.566982,-36.818566,16.936420,46.543585,-80.962171,-146.652218,10.899085,4.370985,4.060272,40.680890,-37.942655,33.838225,1
1,-8.536541,-8.305435,-117.828269,-28.588333,14.223240,-42.087807,-45.538664,58.898976,27.749744,2.545916,-97.233793,-4.899233,-104.382283,2.867261,-128.433975,2.280566,3.412432,31.501910,-47.950176,103.060202,57.027508,121.304258,31.507396,88.282912,66.766185,-4.363974,-5.791376,-32.101939,-52.255449,-4.826111,1
2,126.229973,133.463504,-105.511797,-149.110267,-3.673355,-104.302244,5.815395,100.838385,-46.240211,0.141700,194.054804,-3.134316,-145.510845,86.972373,-126.200888,0.598976,3.351103,51.084960,141.443443,-150.020696,4.819849,88.905713,51.475105,-73.257358,108.947287,8.567240,0.116269,-117.205053,-107.835928,-93.990332,0
3,369.571563,11.850181,-299.969407,29.371721,-3.457523,-115.901854,159.134323,-149.741411,-108.847522,0.078277,-215.802195,-1.724561,-128.050861,-99.997710,138.824836,0.460472,1.891893,-73.956562,-116.557681,310.894782,80.930568,-166.476192,-73.882682,108.222355,19.573192,-2.242024,0.515601,-25.218215,121.674228,-16.877368,0
4,-99.563708,-85.166292,-73.363391,-35.357907,5.437025,-54.892519,56.430532,-49.237377,-50.054608,2.586612,-115.997344,-0.058934,97.666751,36.416791,-124.206264,0.638184,3.989474,-23.264695,-92.966637,121.929801,31.682235,209.163125,-22.847302,-62.067337,58.109167,2.862082,0.920341,-132.875142,3.492953,-15.799596,1


In [6]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,12.844797,-0.299114,17.445400,-1.089765,0.060510,-0.067073,2.009180,-28.428487,16.467341,1.349504,21.271771,0.009568,-12.566037,0.868286,17.187938,1.077477,3.061034,-13.037922,0.671168,21.573813,0.211515,-2.636170,-13.042007,0.428028,0.999935,-0.153092,0.493480,1.096011,0.690934,-8.751687,0.505100
std,331.218970,87.840630,166.830889,77.132117,7.079223,95.328230,112.574723,124.090562,141.363732,1.263255,181.467439,2.024503,87.855783,170.544586,117.722386,0.743995,0.846151,62.050977,80.674359,229.410302,46.418509,165.031271,62.060615,82.821135,77.482044,8.557124,1.888925,99.041787,100.623747,78.557761,0.499999
min,-1625.085835,-285.272638,-670.544061,-287.536745,-25.612006,-404.876372,-395.781633,-514.417062,-526.193023,0.000011,-779.558716,-8.085427,-333.520093,-619.878237,-420.900247,0.001132,0.098466,-254.889907,-314.385808,-819.812924,-196.452972,-766.155471,-255.701616,-362.646319,-276.221505,-31.560708,-9.925537,-381.244544,-388.045812,-338.939731,0.000000
25%,-186.892257,-60.304200,-90.723945,-52.634237,-4.781103,-65.870246,-74.324525,-110.342167,-78.871151,0.073837,-93.345528,-1.365797,-71.433708,-113.590347,-61.817411,0.490904,2.449836,-54.146848,-53.979555,-132.440268,-31.016369,-115.622198,-54.202381,-56.184700,-51.076999,-5.897427,0.045169,-65.880474,-66.301084,-61.817245,0.000000
50%,30.376174,-0.576299,18.361277,-1.449727,-0.017278,-1.396145,2.512421,-27.857590,14.174901,2.525594,24.600206,0.005653,-13.759096,-5.891067,15.790737,0.885891,3.049842,-12.724674,0.733288,17.014555,0.459627,-8.188229,-12.628320,0.334218,0.464923,-0.101883,0.492704,0.700402,-4.469156,-10.324727,1.000000
75%,231.413537,58.639722,126.733466,51.377692,4.827844,63.905144,76.779477,55.226946,110.592913,2.600066,137.302646,1.375353,43.800738,111.017308,95.642441,1.618811,3.747585,28.655681,54.538959,172.038852,31.697139,101.651604,28.586541,55.119716,52.929229,5.634585,0.941508,67.585794,62.984446,43.972321,1.000000
max,1746.951115,348.103285,786.045193,305.712632,28.931791,351.851613,447.816362,402.650987,598.634382,2.674410,870.338426,8.354485,378.617520,630.578879,465.070306,3.785537,4.988274,202.886675,291.272532,1038.727492,191.379527,681.156385,202.958195,302.583135,264.326359,33.644683,12.157634,418.514443,455.981845,340.571609,1.000000


In [7]:
# Матрица корреляций
def corr_plot_matrix(data):
    rs = np.random.RandomState(0)
    df = pd.DataFrame(rs.rand(10, 10))
    corr = data.corr()
    return corr.style.background_gradient(cmap='coolwarm')
    
corr_plot_matrix(train)
# 'RdBu_r' & 'BrBG' are other good diverging colormaps

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,target
0,1.000000,0.000109,-0.030867,0.009952,0.004175,-0.683391,-0.266068,-0.109221,0.125317,-0.035847,-0.142056,0.008859,-0.078458,-0.202917,-0.176205,-0.026825,-0.026203,-0.109527,-0.013128,0.092882,0.003928,-0.445438,-0.109664,0.000327,-0.006170,-0.012697,0.014113,0.010602,0.447847,-0.605029,-0.035295
1,0.000109,1.000000,-0.001406,-0.016535,-0.009035,-0.001497,-0.000011,-0.010821,0.008760,-0.000665,0.004967,0.010632,-0.004129,0.004742,-0.001421,0.000626,-0.001475,-0.010776,-0.009185,0.005533,0.003385,-0.004401,-0.010869,0.014043,-0.010649,0.013245,0.004834,-0.011425,0.004771,0.008732,-0.001111
2,-0.030867,-0.001406,1.000000,0.012963,-0.003464,0.496363,-0.398614,-0.050128,0.083837,-0.116605,0.720349,-0.002068,0.220896,-0.018842,-0.032872,-0.088787,-0.079655,-0.051365,0.013551,-0.569886,-0.014071,0.077907,-0.051319,-0.001905,-0.011012,-0.005426,0.002545,0.000869,0.116138,-0.376552,-0.116546
3,0.009952,-0.016535,0.012963,1.000000,-0.039265,-0.004699,-0.012833,0.007797,0.005755,-0.001657,0.000028,0.001738,0.018217,0.000658,-0.026680,0.001058,-0.014914,0.007706,-0.009971,-0.008084,0.007039,0.008668,0.007859,-0.009039,-0.002669,-0.003955,0.007364,0.005307,0.011472,-0.009545,-0.001668
4,0.004175,-0.009035,-0.003464,-0.039265,1.000000,-0.000014,-0.005950,0.003691,0.007250,-0.002627,0.001977,0.000532,-0.007873,-0.004014,-0.016123,0.003210,0.000216,0.003642,0.001854,-0.000216,-0.003414,-0.003477,0.003699,-0.011019,-0.019228,0.012436,-0.004523,0.000700,0.008240,0.011029,-0.002522
5,-0.683391,-0.001497,0.496363,-0.004699,-0.000014,1.000000,0.145428,0.109130,-0.341398,-0.006321,0.319398,-0.007222,-0.026515,-0.167434,0.188306,-0.006641,-0.000930,0.108992,0.008690,-0.499898,-0.008170,0.036470,0.109080,-0.004696,0.005215,-0.003337,-0.003993,-0.009302,-0.049944,0.283748,-0.006273
6,-0.266068,-0.000011,-0.398614,-0.012833,-0.005950,0.145428,1.000000,-0.012084,-0.210825,-0.006717,-0.309888,-0.011258,0.056663,-0.234035,0.189994,-0.004420,-0.003167,-0.012151,0.006065,0.126850,0.014749,-0.000373,-0.012157,0.000594,0.006739,-0.001582,-0.009197,-0.003050,0.038648,0.063131,-0.006675
7,-0.109221,-0.010821,-0.050128,0.007797,0.003691,0.109130,-0.012084,1.000000,-0.057508,0.012295,-0.221794,0.015388,-0.311304,0.349756,-0.084348,0.005253,-0.002455,0.999937,0.005667,-0.469622,-0.015008,0.040982,0.999937,-0.011824,-0.003211,0.001452,0.003215,0.005726,0.187494,-0.031961,0.012275
8,0.125317,0.008760,0.083837,0.005755,0.007250,-0.341398,-0.210825,-0.057508,1.000000,-0.072142,0.015205,0.008310,0.059753,0.230501,-0.078595,-0.052979,-0.061530,-0.058249,-0.005641,0.463525,0.004253,0.276823,-0.058257,-0.016238,-0.026876,0.004635,-0.001281,-0.000089,-0.188670,-0.026726,-0.072368
9,-0.035847,-0.000665,-0.116605,-0.001657,-0.002627,-0.006321,-0.006717,0.012295,-0.072142,1.000000,-0.085410,-0.010009,-0.086256,0.015857,-0.100750,0.726550,0.735583,0.022478,-0.005644,0.024874,0.004448,-0.001989,0.022533,-0.014408,0.012329,-0.006372,-0.001219,-0.005576,0.008574,0.094110,0.999408


In [8]:
corr_matrix = train.corr().abs()

# Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
#                                   k=1).astype(np.bool))
low = corr_matrix.where(np.tril(np.ones(corr_matrix.shape),
                                  k=-1).astype(np.bool))
# print(); print(upper)

# Find index of feature columns with correlation greater than 0.95
# to_drop2 = [column for column in low.columns if any(low[column] > 0.71)]
to_drop2 = [9]
final_columns_list = train.columns[[i not in to_drop2 for i in train.columns]]
print(); print(to_drop2)
print(); print(final_columns_list)


[9]

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 'target'], dtype='object')


In [9]:
from scipy import stats
train[(np.abs(stats.zscore(train)) >= 1).all(axis=1)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,target


In [10]:
# report = ProfileReport(train)
# report.to_file('profile_report.html')

In [11]:
# import seaborn as sns
# sns.set(style="ticks")
# sns.pairplot(train, hue="target")

In [12]:
# roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True,
#                              needs_threshold=True)
roc_auc_scorer = make_scorer(roc_auc_score, greater_is_better=True)
def pred_GradientBoostingClassifier(data_all,scor=roc_auc_scorer): 
    data_all = data_all.sample(random_state=241, frac=1)    
    all_subject_X=data_all.loc[:, data_all.columns != 'target']
    all_subject_Y=data_all[['target'] ]    
    stat_scores_final=pd.DataFrame()
    clf_stat=[]
    Y=all_subject_Y
    x_train,x_test,y_train,y_test = train_test_split(all_subject_X, Y, test_size=0.3, random_state=240,stratify=Y)
    grid={"n_neighbors": range(1,20,2),
            "weights": ["distance", "uniform"],
            "algorithm": ["ball_tree", "kd_tree", "brute"],
            "leaf_size":[20,30,35,40],
            "p": [1,2]
        }
#     pipe = imbPipeline(steps=[('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])
    clf =  KNeighborsClassifier()
    #Определим лучшие параметры. 
    BP=bestParam_no_scale(x_train,y_train,clf,grid,scor)
    clf_stat=BP[2]
    clf_stat=BP[2].fit(x_train,y_train)
    print(classification_report(y_test,clf_stat.predict(x_test)))
    print('ROC-AUC-Train', roc_auc_score(y_train,clf_stat.predict(x_train)))
    print('ROC-AUC-Test', roc_auc_score(y_test,clf_stat.predict(x_test)))
    print(BP)
    print('общая точность на тесте',clf_stat.score(x_test,y_test))
    churn, active, total,precision=Goal_prediction(x_test,y_test,clf_stat)
    print('\033[1mДоля правильно определенных отток тест:\033[0m',churn)
    print('\033[1mДоля правильно определенных не отток тест:\033[0m',active)
    print("\033[1mТочность в целом тест:\033[0m",total)
#     joblib.dump(clf_stat, 'model_ALL_GBS'+'_part_HW_'+str(number_selection)+'.sav', compress=1)
    return clf_stat,churn,precision



# функция для выбора лучших парамтеров классификатора
def bestParam_no_scale(X,Y,clf,grid, scor):
#     cv = KFold(n_splits=5, shuffle=True, random_state=241)
    cv = StratifiedKFold(n_splits=3,random_state=241, shuffle=True)
#     gs = RandomizedSearchCV(clf, param_distributions=grid, scoring=scor, cv=cv,n_iter=50,
#                             n_jobs=4,verbose=5, random_state=1001)
    gs = GridSearchCV(clf, grid, scoring=scor, cv=cv)
    gs.fit(X, Y.values.ravel())
    return format(gs.best_score_), gs.best_params_, gs.best_estimator_

#Функция для расчета точности
def Goal_prediction(X,Y,clf):
    """Рассчитывает общую точность и точность по каждому из классов"""
    tree_pd=pd.DataFrame(clf.predict(X),index=Y.index,columns=['прогноз'])
    dat1=pd.concat([Y,tree_pd],axis=1)
    dat1.columns=['факт','прогноз']
    churn=dat1[(dat1['факт']==0)&(dat1['прогноз']==0)].shape[0]/dat1[(dat1['факт']==0)].shape[0]
    precision=dat1[(dat1['факт']==0)&(dat1['прогноз']==0)].shape[0]/dat1[(dat1['прогноз']==0)].shape[0]
    active=dat1[(dat1['факт']==1)&(dat1['прогноз']==1)].shape[0]/dat1[(dat1['факт']==1)].shape[0]
    total=dat1[dat1['факт']==dat1['прогноз']].shape[0]/dat1.shape[0]
#     print('Доля правильно определенных отток:',reached)
#     print('Доля правильно определенных не отток:',no_reach)
#     print("\033[1m"+'Точность в целом:'+"\033[0;0m",total)
    return churn, active, total,precision

In [13]:
clf_GBC=pred_GradientBoostingClassifier(train[final_columns_list])
clf_1,churn_train,precision_train=clf_GBC

<ipython-input-12-b47391b12e4e>:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf_stat=BP[2].fit(x_train,y_train)


              precision    recall  f1-score   support

           0       0.78      0.71      0.74      1485
           1       0.74      0.80      0.77      1515

    accuracy                           0.76      3000
   macro avg       0.76      0.76      0.76      3000
weighted avg       0.76      0.76      0.76      3000

ROC-AUC-Train 1.0
ROC-AUC-Test 0.7562089542287562
('0.7516803263640653', {'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 15, 'p': 2, 'weights': 'distance'}, KNeighborsClassifier(algorithm='ball_tree', leaf_size=20, n_neighbors=15,
                     weights='distance'))
общая точность на тесте 0.7566666666666667
Доля правильно определенных отток тест: 0.7104377104377104
Доля правильно определенных не отток тест: 0.801980198019802
Точность в целом тест: 0.7566666666666667


In [14]:
clf_1.classes_

array([0, 1], dtype=int64)

In [15]:
clf_1.fit(train.loc[:,[i not in to_drop2 for i in train.columns]&( train.columns != 'target')],train[['target'] ].values.ravel() )

KNeighborsClassifier(algorithm='ball_tree', leaf_size=20, n_neighbors=15,
                     weights='distance')

In [16]:
test = read_file('test.csv')
test.head()

Размер набора данных: (2000, 30)
Столбцы с Пустыми значений: Series([], dtype: int64)
Типы данных в наборе: {dtype('float64')}


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,394.650101,-102.778703,-48.854416,17.463193,4.744214,-90.504260,135.111157,-27.691507,192.961067,1.755201,-213.139157,-1.371032,167.220896,-438.697273,117.776726,2.678264,2.336249,2.098611,-131.893003,439.386126,-56.947165,140.952506,1.885744,11.483086,-131.665132,12.876561,0.156747,-185.849095,-29.314711,-84.254441
1,62.200701,-139.323500,235.644493,10.860594,1.921110,39.088440,-186.024836,-53.158605,-122.290561,1.426944,235.208582,1.730758,30.691925,232.770741,-150.823103,2.363756,0.432066,0.924914,-5.622402,-402.651882,-45.854744,-148.360929,2.614495,-24.895114,-40.050598,7.376349,1.962719,-123.776910,65.756136,-10.378203
2,232.736729,5.502143,204.738165,45.738959,1.338264,33.604743,92.235943,-115.127927,-95.263085,2.048801,189.115207,0.951928,-47.048687,80.408480,130.937671,1.001301,2.613453,2.824269,-84.963692,-264.639656,-15.638165,-234.606490,0.409558,286.011182,48.109151,-9.277772,1.838195,19.461847,126.144518,-121.300353
3,340.045437,-129.924115,62.694027,61.899097,-1.294288,-58.616589,-55.080664,10.115134,9.051856,2.732693,-21.916455,1.305913,11.176007,44.407738,-27.797463,0.696783,2.966355,0.467539,-72.113602,-56.165896,13.407883,-56.332212,2.801811,70.875892,26.673509,2.389621,2.188665,109.700845,109.378365,-58.425305
4,-60.576957,71.958472,489.203093,121.380719,-3.620431,44.666983,-161.487192,137.920737,540.680255,1.172919,272.675402,0.539000,-9.820108,577.827262,-235.411444,0.808339,2.801212,0.059909,-93.998931,-369.641090,-51.559846,-6.645415,1.889414,-21.814728,-91.701674,-9.206418,1.211916,-11.602746,-8.162451,-87.902970


In [17]:
result = pd.DataFrame(clf_1.predict_proba(test.loc[:,[i not in to_drop2 for i in test.columns]&( test.columns != 'target')])[:,1])
result.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(self.values, np.asarray(other))


,0
0,0.333683
1,0.374770
2,0.341715
3,0.343256
4,0.199610


In [19]:
# Выгрузка итогового набора данных
result.to_csv('output_knn_fs_pipe_gs_9.csv', header=False, index = False )